# 3. Preparación

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 100)

In [3]:
df_original = pd.read_csv('Datasets/listings_summary.csv')
df_original.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2015,https://www.airbnb.com/rooms/2015,20181107122246,2018-11-07,Berlin-Mitte Value! Quiet courtyard/very central,Great location! 30 of 75 sq meters. This wood...,A+++ location! This „Einliegerwohnung“ is an e...,Great location! 30 of 75 sq meters. This wood...,none,It is located in the former East Berlin area o...,...,t,NaN,NaN,f,f,strict_14_with_grace_period,f,f,4,3.76
1,2695,https://www.airbnb.com/rooms/2695,20181107122246,2018-11-07,Prenzlauer Berg close to Mauerpark,NaN,In the summertime we are spending most of our ...,In the summertime we are spending most of our ...,none,NaN,...,t,NaN,NaN,f,f,flexible,f,f,1,1.42


In [4]:
# eliminamos urls
exclude_columns = []
exclude_columns = list(df_original.filter(regex='_url').columns)
# eliminamos id's
exclude_columns += ['scrape_id', 'host_id', "id",]

# variables por observacion
df = df_original.drop(columns=exclude_columns)
exclude_columns += [ 'last_scraped', "calendar_last_scraped",]
exclude_columns += [ 'amenities',"host_name" ]


## variables por observacion duplicadas
exclude_columns += [ 'street',"host_verifications",'country_code','host_location','neighbourhood','host_neighbourhood','host_location','zipcode','market','smart_location' ]
exclude_columns += [ 'host_listings_count', ]

df = df_original.drop(columns=exclude_columns)

In [5]:
# set variables dicotomicas
bool_columns = [
                "is_business_travel_ready",
                "require_guest_profile_picture",
                "require_guest_phone_verification",
                "instant_bookable",
                "requires_license",
                "is_location_exact",
                "host_identity_verified",
                "host_is_superhost",
                "host_has_profile_pic",
                "has_availability"
               ]

for column in bool_columns:
    df[column] = df[column].map({'f':0,'t':1})
    df[column] = pd.to_numeric(df[column],errors='coerce')


In [6]:
# set variables tipo fecha
date_columns = [
                "first_review",
                "last_review",
                "host_since",
               ]

for column in date_columns:
    df[column] = pd.to_datetime(df[column],errors='coerce')

In [7]:
# variables sinteticas (has_a)
has_a_columns = [
                "summary",
                "space",
                "description",
                "neighborhood_overview",
                "notes",
                "transit",
                "access",
                "interaction",
                "house_rules",
                "host_about",
               ]

for column in has_a_columns:
    df["has_a_"+column] = df[column].where(df[column].isnull(), other=1)
    df["has_a_"+column].fillna(0, inplace = True)
    df["has_a_"+column] = pd.to_numeric(df["has_a_"+column],errors='raise')
    
    
    

df.drop(columns=has_a_columns, inplace = True)

In [8]:
#forzar numericos

numeric_columns = [
                "host_response_rate",
                "weekly_price",
                "monthly_price",
                "security_deposit",
                "cleaning_fee",
                "extra_people",
                "price",
               ]

for column in numeric_columns:
    df[column] = df[column].str.replace('%', '', regex=False)
    df[column] = df[column].str.replace('$', '', regex=False)
    df[column] = df[column].str.replace(',', '', regex=False)
    
    df[column] = pd.to_numeric(df[column],errors='raise')
    df[column] = df[column].fillna(0)


host_response_rate
weekly_price
monthly_price
security_deposit
cleaning_fee
extra_people
price


,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,accommodates,...,has_a_summary,has_a_space,has_a_description,has_a_neighborhood_overview,has_a_notes,has_a_transit,has_a_access,has_a_interaction,has_a_house_rules,has_a_host_about
count,22552.0000,0.0,22526.0000,22526.0000,22526.0000,22526.0000,22552.0000,22552.0000,22552.0000,22552.0000,...,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000,22552.0000
mean,39.3519,NaN,0.1337,2.3307,0.9975,0.3863,52.5098,13.4061,0.7439,2.6375,...,0.9573,0.6217,0.9910,0.5117,0.3199,0.5780,0.4805,0.4614,0.5077,0.4961
std,47.2086,NaN,0.3403,19.9478,0.0498,0.4869,0.0308,0.0580,0.4365,1.5105,...,0.2022,0.4850,0.0944,0.4999,0.4665,0.4939,0.4996,0.4985,0.5000,0.5000
min,0.0000,NaN,0.0000,0.0000,0.0000,0.0000,52.3458,13.1036,0.0000,1.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,0.0000,NaN,0.0000,1.0000,1.0000,0.0000,52.4891,13.3754,0.0000,2.0000,...,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
50%,0.0000,NaN,0.0000,1.0000,1.0000,0.0000,52.5091,13.4168,1.0000,2.0000,...,1.0000,1.0000,1.0000,1.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000
75%,100.0000,NaN,0.0000,2.0000,1.0000,1.0000,52.5327,13.4393,1.0000,3.0000,...,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
max,100.0000,NaN,1.0000,1676.0000,1.0000,1.0000,52.6517,13.7576,1.0000,16.0000,...,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 71 columns):
name                                22493 non-null object
experiences_offered                 22552 non-null object
host_since                          22526 non-null datetime64[ns]
host_response_time                  9658 non-null object
host_response_rate                  22552 non-null float64
host_acceptance_rate                0 non-null float64
host_is_superhost                   22526 non-null float64
host_total_listings_count           22526 non-null float64
host_has_profile_pic                22526 non-null float64
host_identity_verified              22526 non-null float64
neighbourhood_cleansed              22552 non-null object
neighbourhood_group_cleansed        22552 non-null object
city                                22547 non-null object
state                               22468 non-null object
country                             22552 non-null object
latitude    

In [10]:
# guardar data preparada
df.to_csv("data_preparada.csv")

In [ ]:
# label encoder 

for column in  df.columns:
    if df[column].dtype=="object":
        print("variable: "+column)
        groups = df.groupby([column], group_keys=True).size().to_frame()
        index = 1
        category = []
        if len(groups)<10000000:
            print("codificada: "+column)
            for gr in groups.iterrows():
                category.append({"index": index, "category": gr[0]})
                index += 1
            for cat in category:
                df_original[column] = df_original[column].replace(cat['category'], cat['index'], inplace=False)



In [ ]:
#guardamos nuevo dataframe
df = df_original.drop(columns=exclude_columns)

In [ ]:
df.info()

In [ ]:
#Feature selector that removes all low-variance features.

df_colums_selected = []

for column in df_original_columns:
    if column not in exclude_dates:
        try:
            varianza = df_original[column + '_to_cluster'].var()
        except KeyError:
            varianza = 0
        
        if  varianza > 0:
            df_colums_selected.append(column)
        else:
            print ("Variables exluidas: " + column)

df_colums_selected = [col+"_to_cluster" for col in df_colums_selected]


exclude_columns

In [ ]:
#limpieza basica y crear variables para analisis
try:
    df_original.drop(columns=["_id"],inplace=True)
except:
    pass

df_columns = list(df.columns) 
exclude_dates = []

for column in df_columns:
    
    if df_original[column].dtype == "int" or df_original[column].dtype == "float":
        df_original[column].fillna(0, inplace = True)
        df_original[column + '_to_analisis'] = df_original[column]
    elif df_original[column].dtype == (object):
        df_original[column].fillna("Sin dato", inplace = True)
        df_original[column + '_to_analisis'] = df_original[column]
    elif df_original[column].dtype == "bool":
        df_original[column].fillna(False, inplace = True)
        df_original[column + '_to_analisis'] = df_original[column]
    else :
        df_original[column+'_year'] = df_original[column].dt.year 
        df_original[column+'_month'] = df_original[column].dt.month 
        df_original[column+'_dayofweek'] = df_original[column].dt.dayofweek
        df_original[column+'_hour'] = df_original[column].dt.hour 
        df_original[column+'_year_to_analisis'] = df_original[column].dt.year 
        df_original[column+'_hour_to_analisis'] = df_original[column].dt.hour 
        df_original[column+'_month_to_analisis'] = df_original[column].dt.month 
        df_original[column+'_dayofweek_to_analisis'] = df_original[column].dt.dayofweek 
        exclude_dates.append(column)